Import necessary libraries

In [1]:
import pandas as pd
import pandas_gbq
import matplotlib.pyplot as plt
import seaborn as sns

Set up credentials or project id parameters

In [ ]:
PROJECT_ID: str = "proyectofinal-389001"
pandas_gbq.context.project = PROJECT_ID
pandas_gbq.context.dialect = "standard"

SQL query to Google BigQuery to load the "Metadata" table from the
Google Maps dataset:

In [ ]:
query: str = f"""--sql
SELECT
    *
FROM `{PROJECT_ID}.Google.Metadata`;
"""

gm_meta_df = pd.read_gbq(
    query=query,
    location="us"
)

This is the resulting DataFrame we get from the previous query:

In [ ]:
display(gm_meta_df.info(), gm_meta_df.head())

In [ ]:
gm_meta_df.sort_values(
    by="avg_rating" and "num_of_reviews",
    ascending=False
).head()

In [ ]:
gm_meta_df.describe()

In [ ]:
gm_meta_df["states"].value_counts()

In [ ]:
gm_meta_df["avg_rating"].value_counts()

In [ ]:
gm_meta_df["category"].value_counts()

In [ ]:
display(
    pd.crosstab(
        index=gm_meta_df["states"],
        columns=gm_meta_df["avg_rating"],
        margins=True
    )
)

In [ ]:
display(
    pd.crosstab(
        index=gm_meta_df["states"],
        columns=gm_meta_df["num_of_reviews"],
        margins=True
    )
)

In [ ]:
plt.scatter(
    x=gm_meta_df["avg_rating"],
    y=gm_meta_df["num_of_reviews"]
)
plt.title("Cantidad estimada de ratings por valor")
plt.xlabel("Valor review")
plt.ylabel("Cantidad")
plt.color((0.5, 0.2, 0.2))
plt.show()

In [ ]:
plt.figure(figsize=(10, 8))
plt.title("Valores de reseñas promedio por estado")
plt.xlabel("Estado")
plt.ylabel("Valor de reseñas")
plt.color((0.35, 0.66, 0.4))
sns.lineplot(
    data=gm_meta_df,
    x="states",
    y="avg_rating"
);

In [ ]:
plt.figure(figsize=(10, 8))
plt.title("Cantidad de reseñas por estado")
plt.xlabel("Estado")
plt.ylabel("Cantidad de reseñas")
plt.color((0.2, 0.45, 0.7))
sns.lineplot(
    data=gm_meta_df,
    x="states",
    y="num_of_reviews"
);

A este punto, para realizar un análisis más exhaustivo, es necesario que
las variables categóricas sean "traspasadas" a variables numéricas, lo
que se logra con la clase OrdinalEncoder de scikit-learn.

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

encoder = OrdinalEncoder()

train_df = pd.DataFrame(index=gm_meta_df.index)

for col in gm_meta_df.columns[:-2]:
    print(f"Encoding column {col} in dataframe...")
    encoder.fit(gm_meta_df[[col]])
    train_df[f"{col}-encoded"] = encoder.transform(gm_meta_df[[col]])
    print(f"Done! Column {col}-encoded created.")
else:
    display(
        train_df.info(),
        gm_meta_df.head(),
        train_df.head()
    )
    del col, gm_meta_df

Show a correlation heatmap based on the ordinal encoded DataFrame:

In [ ]:
def corr_heatmap(corr_df: pd.DataFrame) -> None:
    plt.figure(figsize=(11, 9))
    sns.heatmap(
        data=corr_df,
        cbar=True,
        square=False,
        annot=True,
        fmt=".2f",
        annot_kws={"size": 12},
        cmap="coolwarm"
    )
    plt.xticks(rotation=45)
    plt.yticks(rotation=45)
    plt.show()

In [ ]:
corr_heatmap(train_df.corr())

New SQL query to load the "Reviews" table from the Google dataset:

In [ ]:
query: str = f"""--sql
SELECT
    *
FROM `{PROJECT_ID}.Google.Reviews`
LIMIT 10000;
"""

gm_reviews_df = pd.read_gbq(
    query=query,
    location="us"
)

This is the resulting DataFrame:

In [ ]:
display(gm_reviews_df.info(), gm_reviews_df.head())

In [ ]:
gm_reviews_df.sort_values(by="rating", ascending=False).head()

In [ ]:
gm_reviews_df["rating"].value_counts()

In [ ]:
display(
    pd.crosstab(
        index=gm_reviews_df["rating"],
        columns=gm_reviews_df["date"].dt.year,
        margins=True
    )
)